------

LEFTOVERS:   


    1) Tensor board graphs for loss    
    2) Tensor board for network vis   
    3) Is embedding good or not ?
    4) Embedding visualization ?
    5) model perplexity
    
------

In [1]:
%matplotlib inline

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os.path

In [3]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='Untitled3_1_LOG.txt', mode='a')
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

## Data conversation utils

In [4]:
data_file= '/Users/admin/Documents/Anuj/Coding/Warehouse/harry_potter_3.txt'

fp = open(data_file, 'r')
data = fp.read()

print type(data)

print data[:200]

<type 'str'>
Harry Potter and the Prisoner of Azkaban

by J.K. Rowling

CHAPTER ONE

OWL POST

Harry Potter was a highly unusual boy in many ways. For one thing, he
hated the summer holidays more than any


In [5]:
#get the character set of data
chars = list(set(data))
vocab_size = len(chars)

print len(chars), chars

80 ['\n', '\r', '!', ' ', '"', "'", '&', ')', '(', '*', '-', ',', '.', '1', '0', '3', '2', '5', '4', '7', '6', '9', ';', ':', '?', 'A', 'C', 'B', 'E', 'D', 'G', 'F', 'I', 'H', 'K', 'J', 'M', 'L', 'O', 'N', 'Q', 'P', 'S', 'R', 'U', 'T', 'W', 'V', 'Y', 'X', 'Z', '\\', '_', 'a', '`', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'j', 'm', 'l', 'o', 'n', 'q', 'p', 's', 'r', 'u', 't', 'w', 'v', 'y', 'x', 'z']


In [6]:
data_size, char_set_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, char_set_size)


data has 626260 characters, 80 unique.


In [7]:
char_to_ix = {ch:i for i,ch in enumerate(chars)}

In [8]:
ix_to_char = {i:ch for i,ch in enumerate(chars)}

In [9]:
input = 'ABCDabcd'
map(lambda ch: char_to_ix[ch], input)  

[25, 27, 26, 29, 53, 56, 55, 58]

In [10]:
def c2i(input):
    return map(lambda ch: char_to_ix[ch], input)  

In [11]:
def i2c(input):
    return map(lambda ch: ix_to_char[ch], input)  

--------- 

## Prepare Training data
--------- 

raw_data = 'abcdefghijklmonpqrstuvwxyzABCDEGHIJKLMONPQRSTUVWXYZ123456789!@#$%^&*()'

data_len = len(raw_data)
print data_len

seq_length = 3 # number of steps to unroll the RNN for
batch_size = 6

num_examples = (data_len - 1) // seq_length
print num_examples

num_batches = num_examples // batch_size
print num_batches

for i in range(num_batches):
    print "i = %d" %i
    idx = i * batch_size * seq_length
    print "idx = %d" %idx
    
    for j in range(batch_size):
        start_index = idx + (j*seq_length)
        end_index = idx + (j+1)*seq_length
        print "start_index = %d, end_index = %d" %(start_index, end_index)

raw_data.upper()

In [12]:
# Generate data for an epoch, with batches of size batch_size.

def gen_epoch_data(raw_data, batch_size, seq_length=25):
    
    data_len = len(raw_data)
    
    # num_examples is num_of_sequences. each sequence is an example
    num_examples = (data_len - 1) // seq_length
    
    num_batches = num_examples // batch_size 
    
    # intuituion for the abovce 2 lines is - we are to feed sequences. If each seq is of length k and there are N 
    # sequences in each batch - then a batch basically consists of k*N characters; 
    # thus number of batches = data_len // (seq_lenth * batch_size)
    
    # An easier way to see this is : num of data points = data_len // seq_length
    # therefore num of batches = num of data points // batch_size
    
    # to get training data, we iterate over batches/chunks/rows. First we establish from where the batch data starts 
    # imagine data to be a a st line ------------- of length data_len
    # we split it into batches 
    #    ----------------
    #    ----------------
    #    ----------------
    
    # These (^) text split into chunks/rows - num_of_chunks =  data_len // (seq_lenth * batch_size)
    # first get which chuck to pick. This is done by idx = i * batch_size * seq_length
    
    # once u have the right row - each point in batch is a seq,seq, we need to pick num_of_piars = batch_size
    # so iterate over batch_size
    # starting point in a chunk is jth sequence * length of 1 seq
    
    epoch_data = []
    
    for i in range(num_batches):
        batch = []
        idx = i * batch_size * seq_length
        
        for j in range(batch_size):
            start_index = idx + (j*seq_length)
            end_index = idx + (j+1)*seq_length
            input_ = raw_data[start_index : end_index]
            target_ = raw_data[start_index + 1 : end_index +1 ]
            
            batch.append([c2i(input_), c2i(target_)])
            
        epoch_data.append(batch)
        
    return epoch_data

# Test Data Generation 

In [13]:
check_data = gen_epoch_data(raw_data=data, batch_size=32)

In [14]:
print type(check_data), len(check_data)
print len(check_data[0])
print check_data[0] # this is a list of lists of lists

<type 'list'> 782
32
[[[33, 53, 72, 72, 77, 3, 41, 67, 74, 74, 57, 72, 3, 53, 68, 58, 3, 74, 62, 57, 3, 41, 72, 61, 71], [53, 72, 72, 77, 3, 41, 67, 74, 74, 57, 72, 3, 53, 68, 58, 3, 74, 62, 57, 3, 41, 72, 61, 71, 67]], [[67, 68, 57, 72, 3, 67, 60, 3, 25, 79, 63, 53, 56, 53, 68, 1, 0, 1, 0, 56, 77, 3, 35, 12, 34], [68, 57, 72, 3, 67, 60, 3, 25, 79, 63, 53, 56, 53, 68, 1, 0, 1, 0, 56, 77, 3, 35, 12, 34, 12]], [[12, 3, 43, 67, 75, 66, 61, 68, 59, 1, 0, 1, 0, 26, 33, 25, 41, 45, 28, 43, 3, 38, 39, 28, 1], [3, 43, 67, 75, 66, 61, 68, 59, 1, 0, 1, 0, 26, 33, 25, 41, 45, 28, 43, 3, 38, 39, 28, 1, 0]], [[0, 1, 0, 38, 46, 37, 3, 41, 38, 42, 45, 1, 0, 1, 0, 33, 53, 72, 72, 77, 3, 41, 67, 74, 74], [1, 0, 38, 46, 37, 3, 41, 38, 42, 45, 1, 0, 1, 0, 33, 53, 72, 72, 77, 3, 41, 67, 74, 74, 57]], [[57, 72, 3, 75, 53, 71, 3, 53, 3, 62, 61, 59, 62, 66, 77, 3, 73, 68, 73, 71, 73, 53, 66, 3, 56], [72, 3, 75, 53, 71, 3, 53, 3, 62, 61, 59, 62, 66, 77, 3, 73, 68, 73, 71, 73, 53, 66, 3, 56, 67]], [[67, 77, 3,

# Set Params

In [15]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 2e-1
batch_size = 50


#num_epochs = 500
num_epochs = 1

# Build the Network

In [16]:
import tensorflow as tf

# Input
x = tf.placeholder(tf.int32, shape=(seq_length), name="x")
y = tf.placeholder(tf.int32, shape=(seq_length), name="y")


In [17]:
# One Hot representation of the input
x_oh = tf.one_hot(indices=x, depth=vocab_size)
y_oh = tf.one_hot(indices=y, depth=vocab_size)

In [18]:
#check and balances 

print x.get_shape()
print y.get_shape()

print x_oh.get_shape()
print y_oh.get_shape()

(25,)
(25,)
(25, 80)
(25, 80)


In [19]:
# input and output as of now is 25 characters, each in 1-hot format of length 80
# we will need to feed it 1 character (in 1-hot format) at a time   

#We will convert it into desired format in 2 steps - from (25,80) to 25 tensors of
#shape (80,) then to 25 tensors of shape (1,80)

# Convert the VxN tensor into list of V tensors each of shape = (V,).
rnn_inputs = tf.unpack(value=x_oh)
rnn_targets = tf.unpack(value=y_oh)

#Unpacks the given dimension of a rank=R tensor into rank=(R-1) tensors
#i'th tensor in output is the slice value[i, :]. 
# given a tensor of shape (A, B, C, D);
#If axis == 0 (default value) then the i'th tensor in output is the slice value[i, :, :, :] 
#and return a list of tensors, where each tensor in output will have shape (B, C, D)

# for our data, rank of x_oh is 2 and shape = (25,80)
#rank is number of dimensions


In [30]:
rnn_inputs[0].get_shape()

TensorShape([Dimension(80)])

In [20]:
# Setup the weights and biases.
with tf.variable_scope('rnn_cell'):
    
    Wxh = tf.Variable(tf.truncated_normal(shape=(hidden_size, vocab_size)), name='Wxh')
    Whh = tf.Variable(tf.truncated_normal(shape=(hidden_size, hidden_size)), name='Whh')
    Why = tf.Variable(tf.truncated_normal(shape=(vocab_size, hidden_size)), name='Why')    
    bh = tf.Variable(tf.zeros(shape=(hidden_size, 1)), name='bh')
    by = tf.Variable(tf.zeros(shape=(vocab_size,1)),name='by')

In [21]:

def rnn_cell(rnn_input, cur_state):
    
    # expand_dims is required to make the input a 2-D tensor.
    # Given a tensor input, this operation inserts a dimension of 1 at the dimension 
    # index axis of input's shape. The dimension index axis starts at zero;
    
    with tf.variable_scope('rnn_cell'):
        
        inp = tf.expand_dims(input=rnn_input, axis=1)

        next_state = tf.tanh(tf.matmul(Wxh, inp) + tf.matmul(Whh, cur_state) + bh)
        y_hat = tf.matmul(Why, next_state) + by
    
    return y_hat, next_state

In [22]:
by.get_shape()

TensorShape([Dimension(80), Dimension(1)])

In [23]:
logits = []

#initialise state
state = tf.zeros([hidden_size, 1])

# Iterate over all the input vectors
for rnn_input in rnn_inputs:
    y_hat, state = rnn_cell(rnn_input, state)
    # Convert y_hat into a 1-D tensor
    y_hat = tf.squeeze(y_hat)
    logits.append(y_hat)

In [24]:
# Use the helper method to compute the softmax losses
# (It basically compares the outputs to the expected output)
for logit, target in zip(logits, rnn_targets):
    losses = [tf.nn.softmax_cross_entropy_with_logits(logit, target)] 
              
# Compute the average loss over the batch
total_loss = tf.reduce_mean(losses)

In [25]:
# Under the hood, the operation below computes the gradients and does the backprop!
optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(total_loss)



In [26]:
init_state = tf.zeros([hidden_size, 1])
sample_state = init_state

seed = tf.placeholder(tf.int32, [1], name='seed')
rnn_input = tf.one_hot(seed, vocab_size)
rnn_input = tf.squeeze(rnn_input)

y_hat, sample_state = rnn_cell(rnn_input, sample_state)

prob = tf.nn.softmax(tf.squeeze(y_hat)) 

# Network defined, lets train it

In [27]:

epoch_data = gen_epoch_data(data, batch_size)

tlosses = []

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter('./Char_Model.log', sess.graph)
    
    for epoch_idx in range(num_epochs):
        
        print '--- Starting Epoch:', epoch_idx, '---'
        epoch_loss = 0
        #epoch_state = np.zeros([hidden_size, 1])
        equals = 0.0
        
        for idx, batch in enumerate(epoch_data):
            
            training_loss = 0
            for example_idx, example in enumerate(batch):
                
                x_i = example[0]
                y_i = example[1]
                
                #loss, tloss, _, logits_, rnn_targets_, epoch_state = \
                #            sess.run([losses, total_loss, train_step, logits, rnn_targets, state], \
                #            feed_dict={x:x_i, y:y_i})
                
                loss, tloss, _, logits_, rnn_targets_ = \
                            sess.run([losses, total_loss, optimizer, logits, rnn_targets], \
                            feed_dict={x:x_i, y:y_i})
                    
                    
                    
                _, tloss,  = sess.run([optimizer, total_loss, logits, rnn_targets], \
                            feed_dict={x:x_i, y:y_i})
                    
                    
                    
                logits_argmax = np.argmax(logits_, axis=1)
                rnn_targets_argmax = np.argmax(rnn_targets_, axis=1)
                equals += np.sum(logits_argmax == rnn_targets_argmax)
                training_loss += tloss
                
                if (example_idx % 500 == 0):
                    inp_seed = np.array([example[0][0]])

                    #print '\n'
                    #print '--- SAMPLE BEGIN ---'
                    
                    logging.debug("\n")
                    logging.debug("--- SAMPLE BEGIN ---")
                    
                    num_chars = 100
                    ixes = []
                    sstate = np.zeros([hidden_size, 1])
                    for j in range(num_chars):
                        prob_r, sstate = sess.run([prob, sample_state], feed_dict={seed:inp_seed, init_state:sstate, x:x_i})
                        ix = np.random.choice(vocab_size, p=prob_r.ravel())
                        ixes.append(ix)
                        inp_seed = np.array([ix])

                    #print ''.join(i2c(ixes))
                    #print '--- SAMPLE END ---\n'
                    sent = ''.join(i2c(ixes))
                    logging.debug(sent)
                    logging.debug("--- SAMPLE END ---\n")
                
            training_loss /= len(batch)
            equals /= len(batch)
            print 'Epoch:', epoch_idx, 'Batch:', idx
            print 'Average training loss in batch:', training_loss
            print 'Average matching chars per batch:', equals
            tlosses.append(training_loss)
                

--- Starting Epoch: 0 ---
Epoch: 0 Batch: 0
Average training loss in batch: 18.5773375893
Average matching chars per batch: 0.3
Epoch: 0 Batch: 1
Average training loss in batch: 18.0642033322
Average matching chars per batch: 0.326
Epoch: 0 Batch: 2
Average training loss in batch: 18.374355007
Average matching chars per batch: 0.44652
Epoch: 0 Batch: 3
Average training loss in batch: 22.2926209545
Average matching chars per batch: 0.3489304
Epoch: 0 Batch: 4
Average training loss in batch: 19.9001151323
Average matching chars per batch: 0.426978608
Epoch: 0 Batch: 5
Average training loss in batch: 20.8047089863
Average matching chars per batch: 0.30853957216
Epoch: 0 Batch: 6
Average training loss in batch: 22.0076814159
Average matching chars per batch: 0.526170791443
Epoch: 0 Batch: 7
Average training loss in batch: 21.8963731694
Average matching chars per batch: 0.490523415829
Epoch: 0 Batch: 8
Average training loss in batch: 20.2105157658
Average matching chars per batch: 0.5698104

KeyboardInterrupt: 

In [ ]:
tlosses = train()
plt.plot(tlosses)
plt.show()



In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os.path

data_file= '/Users/admin/Documents/Anuj/Coding/Warehouse/input.txt'

data = open(data_file, 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)

char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 2e-1
batch_size = 50
num_epochs = 500

In [ ]:
# Convert an array of chars to array of vocab indices
def c2i(inp):
    return map(lambda c:char_to_ix[c], inp)

def i2c(inp):
    return map(lambda c:ix_to_char[c], inp)

# Generate data for an epoch, with batches of size batch_size.
def gen_epoch_data(raw_data, batch_size):
    data_len = len(raw_data)
    num_examples = (data_len - 1) // seq_length
    num_batches = num_examples // batch_size

    epoch_data = []
    for i in range(num_batches):
        batch = []
        idx = i * batch_size * seq_length
        for j in range(batch_size):
            inp = raw_data[idx + j*seq_length:idx + (j+1)*seq_length]
            target = raw_data[idx + 1+(j*seq_length):idx + 1+((j+1)*seq_length)]

            batch.append([c2i(inp), c2i(target)])
        epoch_data.append(batch)
    return epoch_data


In [ ]:
epoch_data = gen_epoch_data(data, batch_size)
init_state = tf.zeros([hidden_size, 1])

# Input
x = tf.placeholder(tf.int32, shape=(seq_length), name="x")
y = tf.placeholder(tf.int32, shape=(seq_length), name="y")
state = tf.zeros([hidden_size, 1])

# One Hot representation of the input
x_oh = tf.one_hot(indices=x, depth=vocab_size)
y_oh = tf.one_hot(indices=y, depth=vocab_size)

rnn_inputs = tf.unpack(x_oh)
rnn_targets = tf.unpack(y_oh)


In [ ]:
# Setup the weights and biases.
with tf.variable_scope('rnn_cell'):
    Wxh = tf.get_variable('Wxh', [hidden_size, vocab_size])
    Whh = tf.get_variable('Whh', [hidden_size, hidden_size])
    Why = tf.get_variable('Why', [vocab_size, hidden_size])
    bh = tf.get_variable('bh', [hidden_size, 1])
    by = tf.get_variable('by', [vocab_size, 1])

# Actual math behind computing the output and the next state of the RNN.
def rnn_cell(rnn_input, cur_state):
    with tf.variable_scope('rnn_cell', reuse=True):
        Wxh = tf.get_variable('Wxh', [hidden_size, vocab_size])
        Whh = tf.get_variable('Whh', [hidden_size, hidden_size])
        Why = tf.get_variable('Why', [vocab_size, hidden_size])
        bh = tf.get_variable('bh', [hidden_size, 1])
        by = tf.get_variable('by', [vocab_size, 1])
    inp = tf.expand_dims(rnn_input, 1)

    next_state = tf.tanh(tf.matmul(Wxh, inp) + tf.matmul(Whh, cur_state) + bh)
    y_hat = tf.matmul(Why, next_state) + by
    return y_hat, next_state

In [ ]:
logits = []
for rnn_input in rnn_inputs:
    y_hat, state = rnn_cell(rnn_input, state)
    y_hat = tf.squeeze(y_hat)
    logits.append(y_hat)

losses = [tf.nn.softmax_cross_entropy_with_logits(logit, target) for logit, target in zip(logits, rnn_targets)]
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(total_loss)

num_samples = 25
sample_state = init_state
seed = tf.placeholder(tf.int32, [1], name='seed')
rnn_input = tf.one_hot(seed, vocab_size)
ixes = []

rnn_input = tf.squeeze(rnn_input)
y_hat, sample_state = rnn_cell(rnn_input, sample_state)
prob = tf.nn.softmax(tf.squeeze(y_hat))



In [ ]:
def train():
    
    tlosses = []
    
    #save the model
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        if os.path.isfile("model.ckpt"):
            saver.restore(sess, "model.ckpt")
        else:
            sess.run(tf.initialize_all_variables())

        for epoch_idx in range(num_epochs):
            print '--- Starting Epoch:', epoch_idx, '---'
            epoch_loss = 0
            epoch_state = np.zeros([hidden_size, 1])
            equals = 0.0
            for idx, batch in enumerate(epoch_data):
                training_loss = 0
                for example_idx, example in enumerate(batch):
                    x_i = example[0]
                    y_i = example[1]

                    loss, tloss, _, logits_, rnn_targets_, epoch_state = \
                        sess.run([losses, total_loss, train_step, logits, \
                            rnn_targets, state], \
                                feed_dict={x:x_i, y:y_i, init_state:epoch_state}
                        )

                    logits_argmax = np.argmax(logits_, axis=1)
                    rnn_targets_argmax = np.argmax(rnn_targets_, axis=1)
                    equals += np.sum(logits_argmax == rnn_targets_argmax)

                    training_loss += tloss

                    if (example_idx % 100 == 0):
                        inp_seed = np.array([example[0][0]])

                        print '\n'
                        print '--- SAMPLE BEGIN ---'
                        num_chars = 100
                        ixes = []
                        sstate = np.zeros([hidden_size, 1])
                        for j in range(num_chars):
                            prob_r, sstate = sess.run([prob, sample_state], feed_dict={seed:inp_seed, init_state:sstate, x:x_i})
                            ix = np.random.choice(vocab_size, p=prob_r.ravel())
                            ixes.append(ix)
                            inp_seed = np.array([ix])

                        print ''.join(i2c(ixes))
                        print '--- SAMPLE END ---'

                training_loss /= len(batch)
                equals /= len(batch)
                print 'Epoch:', epoch_idx, 'Batch:', idx
                print 'Average training loss in batch:', training_loss
                print 'Average matching chars per batch:', equals
                tlosses.append(training_loss)
            save_path = saver.save(sess, "model.ckpt")
            print("Model saved in file: %s" % save_path)
    return tlosses


In [ ]:
tlosses = train()
plt.plot(tlosses)
plt.show()